In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import pandas
import numpy as np
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

In [3]:
import pandas
#dataset taken from https://www.kaggle.com/yashsawarn/wifi-stretgth-for-rooms
    
    
def read_dataset(csv_name = 'sao-paulo-properties-april-2019.csv'):
    """
    Reads a csv dataset 
    returns it as a pytorch tensor
    """
    data_frame = pandas.read_csv(csv_name)
        
    #do data preprocessing and return torch matrix with targets in the last column
    """
    Creates a new column with categories using the "price" column
      1) 900000 < y , categoría 4
      2) 580000 < y < 900000, categoría 3
      3) 400000 < y < 580000, categoría 2
      4) y < 400000, categoría 1    
    """
    data_frame.loc[ data_frame['Price'] <  400000, 'Class'] = 1
    data_frame.loc[(data_frame['Price'] >= 400000) & (data_frame['Price'] < 580000), 'Class'] = 2
    data_frame.loc[(data_frame['Price'] >= 580000) & (data_frame['Price'] < 900000), 'Class'] = 3
    data_frame.loc[ data_frame['Price'] >= 900000, 'Class'] = 4
    data_frame['Class'] = data_frame['Class'].astype('int')
    
    #Remove all unnecesary attributes and leave only the ones that we need
    data_frame = data_frame.loc[:, ['Rooms', 'Size', 'Toilets', 'Parking', 'Class']]
    
    return data_frame

In [4]:
class Node_CART:    
    def __init__(self, num_classes = 4, ref_CART = None, current_depth = 0):
        """
        Create the node attributes
        param num_classes: K number of classes to classify
        param ref_cart: reference to the tree containing the node
        param current_depth: current depth of the node in the tree
        """
        self.ref_CART = ref_CART
        self.threshold_value = 0
        self.feature_num = 0
        self.node_right = None
        self.node_left = None
        self.data_torch_partition = None
        self.gini = 0
        self.dominant_class = None
        self.accuracy_dominant_class = None        
        self.num_classes = num_classes
        self.current_depth = current_depth
    
    def to_xml(self, current_str = ""):
        """
        Recursive function to write the node content to an xml formatted string
        param current_str : the xml content so far in the whole tree
        return the string with the node content
        """
        str_node = "<node><thresh>" + str(self.threshold_value) + "</thresh>" + "<feature>" + str(self.feature_num) + "</feature><depth>" + str(self.current_depth)+ "</depth>" 
        str_node += "<gini>" + str(self.gini) + "</gini>"
        if(self.node_right != None):
            str_left = self.node_right.to_xml(current_str)
            str_node += str_left
        if(self.node_left != None):
            str_right = self.node_left.to_xml(current_str)
            str_node += str_right
            
        if(self.is_leaf()):
            str_node += "<dominant_class>" + str(self.dominant_class) + "</dominant_class><acc_dominant_class>"  + str(self.accuracy_dominant_class) + "</acc_dominant_class>"
        str_node += "</node>"
        return str_node
    
    def is_leaf(self):
        """
        Checks whether the node is a leaf
        """
        return (self.node_left == None and self.node_right == None)
    
    def create_with_children(self, data_torch, current_depth, list_selected_features = [], min_gini = 0.000001):
        """
        Creates a node by selecting the best feature and threshold, and if needed, creating its children
        param data_torch: dataset with the current partition to deal with in the node
        param current_depth: depth counter for the node
        param list_selected_features: list of selected features so far for the CART building process
        param min_gini: hyperparmeter selected by the user defining the minimum tolerated gini coefficient for a  node
        return the list of selected features so far
        """        
        #update depth of children
        depth_children = current_depth + 1
        if(depth_children <= self.ref_CART.get_max_depth()):
            num_observations = data_torch.shape[0]            
            #careful with max depth
            #if no threshold and feature were selected, select it using a greedy approach            
            (threshold_value, feature_num, gini) = self.select_best_feature_and_thresh(data_torch, list_features_selected = list_selected_features)
            list_selected_features += [feature_num]
            #store important data in attributes
            self.threshold_value = threshold_value
            self.feature_num = feature_num
            self.data_torch_partition = data_torch
            self.gini = gini            
            num_features = data_torch.shape[1]
            #data_torch_left = torch.zeros(1, num_features)
            #data_torch_right = torch.zeros(1, num_features)
            #create the right and left node data if the current gini is still high            
            if(self.gini > min_gini):
                data_torch_left = data_torch[data_torch[:, feature_num] < threshold_value]
                data_torch_right = data_torch[data_torch[:, feature_num] >= threshold_value]
                #if the new partitions have more than min_observations, make them
                if(data_torch_left.shape[0] >= self.ref_CART.get_min_observations() and data_torch_right.shape[0] >= self.ref_CART.get_min_observations()):
                    #add data to the right and left children
                    self.node_right = Node_CART(num_classes = self.num_classes, ref_CART = self.ref_CART, current_depth = depth_children)
                    self.node_left = Node_CART(num_classes = self.num_classes, ref_CART = self.ref_CART, current_depth = depth_children)
                    list_selected_features = self.node_right.create_with_children(data_torch_right, depth_children, list_selected_features = list_selected_features)            
                    self.node_left.create_with_children( data_torch_left, depth_children, list_selected_features = list_selected_features)
        #if is leaf, fill the         
        if(self.is_leaf()):
            labels_data = data_torch[:, -1]
            self.dominant_class = torch.mode(labels_data).values.item()
            num_obs_label = labels_data[labels_data == self.dominant_class].shape[0]
            self.accuracy_dominant_class = num_obs_label / labels_data.shape[0]           
            
        return list_selected_features
    
    
    def select_best_feature_and_thresh(self, data_torch, list_features_selected = [], num_classes = 4):
        """
        ONLY USE  2 FORS
        Selects the best feature and threshold that minimizes the gini coefficient
        param data_torch: dataset partition to analyze
        param list_features_selected list of features selected so far, thus must be ignored 
        param num_classes: number of K classes to discriminate from 
        return min_thresh, min_feature, min_gini found for the dataset partition when 
        selecting the found feature and threshold
        """       
        min_thresh = 0
        min_feature = ''
        min_gini = 1
        
        for feature_num in range(num_classes):
            for tresh in data_torch[:, feature_num].unique():
                data_torch_left = data_torch[data_torch[:, feature_num] < tresh]
                data_torch_right = data_torch[data_torch[:, feature_num] >= tresh]
                
                left_classes, right_classes = data_torch_left[:, -1], data_torch_right[:, -1] 
                
                gini_left = self.calculate_gini(left_classes, num_classes)
                gini_right = self.calculate_gini(right_classes, num_classes)
                gini_ponderado = ((data_torch_left.shape[0] / data_torch.shape[0]) * gini_left) + ((data_torch_right.shape[0] / data_torch.shape[0]) * gini_right)
                
                if gini_ponderado < min_gini:
                    min_gini = gini_ponderado
                    min_feature = feature_num
                    min_thresh = tresh.item()
                
            
        return (min_thresh, min_feature, min_gini)   
        
    
    def calculate_gini(self, data_partition_torch, num_classes = 4):
        """
        Calculates the gini coefficient for a given partition with the given number of classes
        param data_partition_torch: current dataset partition as a tensor
        param num_classes: K number of classes to discriminate from
        returns the calculated gini coefficient
        """
        uniq, counts = data_partition_torch.unique(return_counts=True)
        totalQty = data_partition_torch.shape[0]
        gini = 1 - sum( (counts / totalQty)  **2)
        return gini
   
    
    def evaluate_node(self, input_torch): 
        """
        Evaluates an input observation within the node. 
        If is not a leaf node, send it to the corresponding node
        return predicted label
        """
        feature_val_input = input_torch[self.feature_num]
        if(self.is_leaf()):
            return self.dominant_class
        else:
            if(feature_val_input < self.threshold_value):
                return self.node_left.evaluate_node(input_torch)
            else:
                return self.node_right.evaluate_node(input_torch)
        

class CART:
    def __init__(self, dataset_torch, max_CART_depth = 4, min_observations = 2):
        """
        CART has only one root node
        """
        #min observations per node
        self.min_observations = min_observations
        self.root = Node_CART(num_classes = 4, ref_CART = self, current_depth = 0)
        self.max_CART_depth = max_CART_depth
        self.list_selected_features = []
        
    def get_root(self):
        """
        Gets tree root
        """
        return self.root
    
    def get_min_observations(self):
        """
        return min observations per node
        """
        return self.min_observations
    
    def get_max_depth(self):
        """
        Gets the selected max depth of the tree
        """
        return self.max_CART_depth
    
    def build_CART(self, data_torch):
        """
        Build CART from root
        """
        self.list_selected_features = self.root.create_with_children(data_torch, current_depth = 0)
    
    def to_xml(self, xml_file_name):
        """
        write Xml file with tree content
        """
        str_nodes = self.root.to_xml()
        file = open(xml_file_name,"w+") 
        file.write(str_nodes)
        file.close()
        return str_nodes
    
    
    def evaluate_input(self, input_torch):
        """
        Evaluate a specific input in the tree and get the predicted class
        """
        return self.root.evaluate_node(input_torch)
        
    
def train_CART(dataset_torch, name_xml = "", max_CART_depth = 3, min_obs_per_leaf = 2): 
    """
    Train CART model
    """
    tree = CART(dataset_torch = dataset_torch, max_CART_depth = max_CART_depth, min_observations =  min_obs_per_leaf)
    tree.build_CART(dataset_torch)
    if(not name_xml == ""):
        tree.to_xml(name_xml)
    return tree

def test_CART(tree, testset_torch):
    """
    Test a previously built CART
    """
    correct_observations = 0
    for observation in testset_torch:
        predicted = tree.evaluate_input(observation)
        classy = observation[-1].item()
        if predicted == classy:
            correct_observations += 1 
    return correct_observations / testset_torch.shape[0]

def partition_validation(dataset_torch, max_CART_depth, num_splits):
    rs = ShuffleSplit(n_splits=num_splits, train_size=0.7, test_size=0.3, random_state=0)
    accuracys = []
    for train_index, test_index in rs.split(dataset_torch):
        treex = train_CART(dataset_torch[train_index], name_xml = "CART_depth_partitions.xml", max_CART_depth = max_CART_depth)
        accx = test_CART(treex, dataset_torch[test_index])
        accuracys.append([accx])
    return accuracys
        
        
class Random_Forest:
    """
    Creates the random forest
    param num_trees: K number of trees that will be generated
    param original_data: original dataset
    param random_data_subsets: list of random subsets that will be used in the different CARTs
    param list_of_carts: list of CARTs
    param depth_of_trees: depth of all the tree in the random forest
    """
    def __init__(self, original_data = None, num_trees = 0, depth_of_trees = 2):
        self.original_data = original_data
        self.num_trees = num_trees
        self.random_data_subsets = []
        self.list_of_carts = []
        self.depth_of_trees = depth_of_trees
    
        
    def num_trees(self):
        """
        Gets the num of CARTs
        """
        return self.num_trees
    
    def get_original_data(self):
        """
        Gets the original dataset
        """
        return self.original_data
    
    def get_random_data_subsets(self):
        """
        Gets a list of the random subsets that will generate the CARTs
        """
        return self.max_CART_depth
    
    def get_list_of_carts(self):
        """
        Gets a list of the random subsets that will generate the CARTs
        """
        return self.list_of_carts
    
    def get_depth_of_trees(self):
        """
        Gets a list of the random subsets that will generate the CARTs
        """
        return self.depth_of_trees
    
    def generate_random_forest(self, partitions):
        for train_index, test_index in partitions.split(self.original_data):
            self.random_data_subsets.append(self.original_data[train_index])
        idx = 0
        for data_subset in self.random_data_subsets:
            treex = train_CART(data_subset, "Forest_CART" + str(idx) + ".xml", self.depth_of_trees, 2)
            self.list_of_carts.append(treex)
            idx += 1
            
    def evaluate_random_forest(self, input_torch):
        predicted_categories = []
        for cart in self.list_of_carts:
            predicted_categories.append(cart.evaluate_input(input_torch))
        #Returns the most common element from the prediction of all trees
        return max(set(predicted_categories), key=predicted_categories.count)
        
    def test_random_forest(self, testset_torch):
        correct_observations = 0
        for observation in testset_torch:
            predicted = self.evaluate_random_forest(observation)
            classy = observation[-1].item()
            if predicted == classy:
                correct_observations += 1
        return correct_observations / testset_torch.shape[0]
            

def train_random_forest(testset_torch, k_partitions, depth_per_tree):
    forest = Random_Forest(testset_torch, k_partitions, depth_per_tree)
    kf = KFold(n_splits=k_partitions, shuffle=True)
    forest.generate_random_forest(kf)
    return forest


dataset_torch = read_dataset(csv_name="/content/drive/MyDrive/Colab Notebooks/P1/sao-paulo-properties-april-2019.csv")

dataset_torch = torch.tensor(dataset_torch.values)

#### EVALUACION DEL CART

tree2 = train_CART(dataset_torch, name_xml = "CART_depth_2.xml", max_CART_depth = 2)
acc2 = test_CART(tree2, dataset_torch)
print("The accuracy for depth 2 is of: ", acc2, "\n")

tree3 = train_CART(dataset_torch, name_xml = "CART_depth_3.xml", max_CART_depth = 3)
acc3 = test_CART(tree3, dataset_torch)
print("The accuracy for depth 3 is of: ", acc3, "\n")

### PARTICIONES

print("************** Testing with partitions **************** \n")

print("Testing with partitions and depth = 2 \n")

accuracys = partition_validation(dataset_torch, 2, 10)
accurracy_df = pandas.DataFrame(accuracys, columns = ['Accuracy of partition'])
print(accurracy_df)
print("\n The average accuracy is :", accurracy_df['Accuracy of partition'].mean())
print("\n The standard deviation is :", accurracy_df['Accuracy of partition'].std())


print("\n Testing with partitions and depth = 3 \n")

accuracys = partition_validation(dataset_torch, 3, 10)
accurracy_df = pandas.DataFrame(accuracys, columns = ['Accuracy of partition'])
print(accurracy_df)
print("\n The average accuracy is :", accurracy_df['Accuracy of partition'].mean())
print("\n The standard deviation is :", accurracy_df['Accuracy of partition'].std())

#RANDOM FOREST

print("\n GENERATING RANDOM FOREST \n")

rf_model = train_random_forest(dataset_torch, 5, 3)
forest_acc = rf_model.test_random_forest(dataset_torch)

print("Accuracy of the random forest is:", forest_acc)

print("\n***** EVALUATING RANDOM FOREST ******\n")

accuracys = []
print("\n GENERATING RANDOM FOREST OF 3 CARTS - 10 RUNS \n")
for i in range(10):
    rf_model = train_random_forest(dataset_torch, 3, 3)
    forest_acc = rf_model.test_random_forest(dataset_torch)
    accuracys.append([forest_acc])
forest_accurracy_df = pandas.DataFrame(accuracys, columns = ['Accuracy of run'])
print("Accuracys of 10 runs with random forest of 3 CARTS \n", forest_accurracy_df)
print("\n The average accuracy is :", forest_accurracy_df['Accuracy of run'].mean())
print("\n The standard deviation is :", forest_accurracy_df['Accuracy of run'].std())

print("\n GENERATING RANDOM FOREST OF 5 CARTS - 10 RUNS \n")

accuracys = []
for i in range(10):
    rf_model = train_random_forest(dataset_torch, 5, 3)
    forest_acc = rf_model.test_random_forest(dataset_torch)
    accuracys.append([forest_acc])
forest_accurracy_df = pandas.DataFrame(accuracys, columns = ['Accuracy of run'])
print("Accuracys of 10 runs with random forest of 5 CARTS \n", forest_accurracy_df)
print("\n The average accuracy is :", forest_accurracy_df['Accuracy of run'].mean())
print("\n The standard deviation is :", forest_accurracy_df['Accuracy of run'].std())

The accuracy for depth 2 is of:  0.6438496117695137 

The accuracy for depth 3 is of:  0.6442582754393135 

************** Testing with partitions **************** 

Testing with partitions and depth = 2 

   Accuracy of partition
0               0.641253
1               0.618788
2               0.633084
3               0.651464
4               0.638530
5               0.631042
6               0.639891
7               0.653506
8               0.632403
9               0.625596

 The average accuracy is : 0.6365554799183119

 The standard deviation is : 0.010760737115253691

 Testing with partitions and depth = 3 

   Accuracy of partition
0               0.646018
1               0.626957
2               0.633084
3               0.651464
4               0.645337
5               0.634445
6               0.639891
7               0.658952
8               0.632403
9               0.629680

 The average accuracy is : 0.6398230088495576

 The standard deviation is : 0.01037088394750886

 GENER